# This script contains the below chapters:
1. Importing libraries and data
2. Creating price_label column
3. Creating busiest_day column
4. Creating busiest_days column
5. Creating busiest_period_of_day column
6. Export of new file and final checks

# 1. Importing libraries and data

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
# assigning path for easier import of data
path = r'C:\Users\magia\06-2025 Instacart Basket Analysis'

In [3]:
# importing new data set using path variable
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [4]:
# checking data types and dataframe size (after changes made previously)
df_ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 16 columns):
 #   Column               Dtype   
---  ------               -----   
 0   order_id             int32   
 1   user_id              int32   
 2   order_number         int8    
 3   orders_day_of_week   int8    
 4   order_hour_of_day    int8    
 5   days_between_orders  float32 
 6   new_customer         bool    
 7   product_id           int32   
 8   add_to_cart_order    int32   
 9   reordered            int8    
 10  product_name         object  
 11  aisle_id             int8    
 12  department_id        int8    
 13  prices               float64 
 14  _merge               category
 15  price_range          object  
dtypes: bool(1), category(1), float32(1), float64(1), int32(4), int8(6), object(2)
memory usage: 1.6+ GB


# 2. Creating price_label column

In [5]:
# creating subset with first 1 million rows in the dataframe before creating new variables
df = df_ords_prods_merge[:1000000]

In [6]:
# creating price_label function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return np.nan

In [7]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    673198
Low-range product    314392
High range            12410
Name: count, dtype: int64

In [8]:
# checking highest prices within subset
df['prices'].max()

14900.0

In [9]:
# checking which product has suspicious price of 14900.0
df_ords_prods_merge[df_ords_prods_merge['prices'] >= 1000][['product_name', 'prices']].drop_duplicates()

,product_name,prices
1576,Lowfat 2% Milkfat Cottage Cheese,14900.0


In [10]:
# fixing price
df_ords_prods_merge.loc[
    (df_ords_prods_merge['prices'] > 14899.0) & 
    (df_ords_prods_merge['prices'] < 14901.0), 
    'prices'
] = 1.1

In [11]:
# checking if fix worked
(df_ords_prods_merge['prices'] > 14899.0).sum()

0

In [12]:
# updating data set
df_ords_prods_merge['price_range'] = df_ords_prods_merge.apply(price_label, axis=1)

In [13]:
# save cleaned merged data set
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cleaned.pkl'))

In [14]:
# checking that price_range returns only low-and mid-range products
df_ords_prods_merge['price_range'].value_counts(dropna=False)

price_range
Mid-range product    21861558
Low-range product    10130750
High range             412551
Name: count, dtype: int64

In [15]:
# checking prices of suspicious high range products
df_ords_prods_merge[df_ords_prods_merge['price_range'] == 'High range']['prices'].value_counts().sort_index(ascending=False).head(20)

prices
25.000000    13772
24.900000     5781
24.799999      410
24.700001     7283
24.600000     1962
24.500000     2148
24.400000      499
24.299999     2889
24.200001    15891
24.100000      978
24.000000     4327
23.900000     2074
23.799999    12585
23.700001     1346
23.600000     8458
23.500000     1924
23.400000      368
23.299999     4803
23.200001    14866
23.100000     9135
Name: count, dtype: int64

In [16]:
# Round 'prices' column (float32) to 2 decimal places to fix floating-point precision errors
df_ords_prods_merge['prices'] = df_ords_prods_merge['prices'].round(2)

# Reapply the price labeling function to update 'price_range' categories based on cleaned prices
df_ords_prods_merge['price_range'] = df_ords_prods_merge.apply(price_label, axis=1)

# Check the distribution of price ranges to verify changes
print(df_ords_prods_merge['price_range'].value_counts(dropna=False))

price_range
Mid-range product    21861558
Low-range product    10130750
High range             412551
Name: count, dtype: int64


In [17]:
# Check unique suspicious high prices
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 20, 'prices'].unique()

array([23.2, 22.3, 21.1, 24.2, 23. , 23.9, 23.6, 23.1, 20.7, 20.6, 22.2,
       24.1, 20.4, 24.7, 21.6, 22.1, 24. , 23.7, 23.3, 25. , 21.3, 21.7,
       24.5, 23.8, 22.4, 21.8, 24.4, 22.9, 22.6, 24.6, 24.3, 20.1, 21.9,
       23.4, 20.9, 20.5, 22. , 22.8, 20.3, 24.9, 23.5, 24.8, 20.8, 22.5,
       21.5, 21.4, 20.2, 22.7, 21. ])

In [18]:
# Check if any prices greater than 30 remain (just to be safe)
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 30, ['product_name', 'prices']]

,product_name,prices


In [19]:
# Get summary stats of prices in  subset
print(df['prices'].describe())

# Check distribution of price ranges in subset
print(df['price_range'].value_counts(dropna=False))

count    1000000.000000
mean           7.781304
std            4.233667
min            1.000000
25%            4.200000
50%            7.300000
75%           11.300000
max           25.000000
Name: prices, dtype: float64
price_range
Mid-range product    673198
Low-range product    314392
High range            12410
Name: count, dtype: int64


**Recap**: We have in our subset **over 12.000 products** that are in the high-range, but, after fixing extreme/errouneous high prices of 99000 and 149000 we have **realistic prices of up to 25 USD**.

# 3. Creating busiest_day column

In [20]:
# Create 'busiest_day' based on orders_day_of_week
busiest_day_result = []

for value in df_ords_prods_merge['orders_day_of_week']:
    if value == 0:
        busiest_day_result.append("Busiest day")
    elif value == 4:
        busiest_day_result.append("Least busy")
    else:
        busiest_day_result.append("Regularly busy")

df_ords_prods_merge['busiest_day'] = busiest_day_result

In [21]:
# Count each label's occurrences
df_ords_prods_merge['busiest_day'].value_counts(dropna=False)

busiest_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

In [22]:
# Count percentages of each label
df_ords_prods_merge['busiest_day'].value_counts(normalize=True, dropna=False)

busiest_day
Regularly busy    0.691775
Busiest day       0.191458
Least busy        0.116767
Name: proportion, dtype: float64

**Customer Shopping Behavior by Day of the Week:**

- **69%** of orders are placed on **regularly busy days** (Mon–Sat excluding Wed).
- **19%** of orders occur on **Sunday**, the **busiest day**.
- **12%** of orders are made on **Wednesday**, the **least busy day**.

**Insight**: Wednesday sees slightly lower traffic. While the gap is modest (~1–2%), it could still matter at scale. It might be a good candidate for  ** inventory resets, lighter staff schedules**, or **low-cost promotions**.

# 4. Creating busiest_days column (plural)

In [23]:
# Count orders per day (sorted descending)
day_counts = df_ords_prods_merge['orders_day_of_week'].value_counts().sort_values(ascending=False)
print(day_counts)

orders_day_of_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64


**We see that Sunday and Monday are busiest days; Wednesday and Thursday are least busy two days**

In [24]:
# Creating busiest_days column

busiest_days_result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
    if value in [0, 1]:
        busiest_days_result.append("Busiest two days")
    elif value in [3, 4]:
        busiest_days_result.append("Least two busy days")
    else:
        busiest_days_result.append("Regularly busy days")

df_ords_prods_merge['busiest_days'] = busiest_days_result

In [26]:
# Count occurrences of each label in 'busiest_days'
df_ords_prods_merge['busiest_days'].value_counts(dropna=False)

busiest_days
Regularly busy days    12916111
Busiest two days       11864412
Least two busy days     7624336
Name: count, dtype: int64

In [27]:
# Count percentages of each label
df_ords_prods_merge['busiest_days'].value_counts(normalize=True, dropna=False)

busiest_days
Regularly busy days    0.398586
Busiest two days       0.366131
Least two busy days    0.235284
Name: proportion, dtype: float64

**Customer Shopping Behavior by Day of the Week**
- 40% of orders occur on regularly busy days (Tuesday, Thursday, Friday, Saturday).

- 36% of orders are placed on the two busiest days: Sunday and Monday.

- 24% of orders happen on the two slowest days: Wednesday and Thursday.

**Business Insight**:
Tuesday and Wednesday behave like “weekend days” with lower order volume. These days present opportunities to optimize operations by reducing staff hours or scheduling inventory maintenance. Additionally, targeted low-cost promotions on these slower days could help boost traffic and even out demand throughout the week.

# 5. Creating busiest_period_of_day column

In [28]:
# retrieving exact name of column related to hour of the day
df_ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_between_orders', 'new_customer',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', '_merge', 'price_range',
       'busiest_day', 'busiest_days'],
      dtype='object')

In [29]:
# Count orders per hour (sorted in descending order)
hour_counts = df_ords_prods_merge['order_hour_of_day'].value_counts().sort_values(ascending=False)
print(hour_counts)

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64


In [30]:
# creating busiest_period_of_day column
busiest_period_of_day_result = []

for hour in df_ords_prods_merge['order_hour_of_day']:
    if 6 <= hour < 16:
        busiest_period_of_day_result.append("Most orders")
    elif 17 <= hour <= 23:
        busiest_period_of_day_result.append("Average Orders")
    else:
        busiest_period_of_day_result.append("Fewest Orders")

df_ords_prods_merge['busiest_period_of_day'] = busiest_period_of_day_result

In [31]:
# Count occurrences of each label in new column
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna=False)

busiest_period_of_day
Most orders       21482534
Average Orders     7790795
Fewest Orders      3131530
Name: count, dtype: int64

In [32]:
# Count percentages of each label
df_ords_prods_merge['busiest_period_of_day'].value_counts(normalize=True, dropna=False)

busiest_period_of_day
Most orders       0.662942
Average Orders    0.240421
Fewest Orders     0.096638
Name: proportion, dtype: float64

**Notes on orders per period of the day and business recommendations**
-- 
- Most orders (66%) occurr between 7 AM and 4 PM: Ensure infrastructure and staffing are robust to handle peak traffic; consider premium or time-sensitive promotions.

- Late afternoon until bedtime (5 PM to 11 PM) brings almost 25% of order volume and therefore labeled "Average order" as time period. Therefore, a moderate resource allocation is advised.

- Fewest orders (less than 1%) occurr between 12 AM and 6 AM: Minimal staffing needed, and no sense in doing any promotional activity.

# 6. Export of new file and final checks

In [34]:
# exporting data frame with new columns as pickle file
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_new_columns.pkl'))


In [35]:
# Check if all expected columns are present
expected_columns = ['busiest_day', 'busiest_days', 'busiest_period_of_day']
missing_columns = [col for col in expected_columns if col not in df_ords_prods_merge.columns]

if missing_columns:
    print("⚠️ Missing columns:", missing_columns)
else:
    print("✅ All expected new columns are present.")


✅ All expected new columns are present.


In [37]:
# Checking new pickle file
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_new_columns.pkl'))

# Preview the first 5 rows
print(df.head())

# Check columns and shape
print(df.columns)
print(df.shape)

   order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2539329        1             1                   2                  8   
2   2539329        1             1                   2                  8   
3   2539329        1             1                   2                  8   
4   2539329        1             1                   2                  8   

   days_between_orders  new_customer  product_id  add_to_cart_order  \
0                  NaN          True         196                  1   
1                  NaN          True       14084                  2   
2                  NaN          True       12427                  3   
3                  NaN          True       26088                  4   
4                  NaN          True       26405                  5   

   reordered                             product_name  aisle_id  \
0          0                               

In [38]:
#checking size of new pickle file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 19 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int32   
 1   user_id                int32   
 2   order_number           int8    
 3   orders_day_of_week     int8    
 4   order_hour_of_day      int8    
 5   days_between_orders    float32 
 6   new_customer           bool    
 7   product_id             int32   
 8   add_to_cart_order      int32   
 9   reordered              int8    
 10  product_name           object  
 11  aisle_id               int8    
 12  department_id          int8    
 13  prices                 float64 
 14  _merge                 category
 15  price_range            object  
 16  busiest_day            object  
 17  busiest_days           object  
 18  busiest_period_of_day  object  
dtypes: bool(1), category(1), float32(1), float64(1), int32(4), int8(6), object(5)
memory usage: 2.3+ GB
